In [2]:
import torch
import torchtext
from torch.utils.data import Dataset, DataLoader
from torchtext.data.utils import get_tokenizer
from torchtext.vocab import build_vocab_from_iterator
from nltk.tokenize import word_tokenize
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import time

In [4]:
tweet_data_path = '/kaggle/input/cleaned-cb-data/cyberbullying_dataset.csv'
dataset = pd.read_csv(tweet_data_path)

comments = dataset['tweet_text']
labels = dataset['cyberbullying_type']

In [5]:
dataset

,tweet_text,cyberbullying_type
0,"In other words #katandandre, your food was cra...",3
1,Why is #aussietv so white? #MKR #theblock #ImA...,3
2,@XochitlSuckkks a classy whore? Or more red ve...,3
3,"@Jason_Gio meh. :P thanks for the heads up, b...",3
4,@RudhoeEnglish This is an ISIS account pretend...,3
...,...,...
47687,"Black ppl aren't expected to do anything, depe...",1
47688,Turner did not withhold his disappointment. Tu...,1
47689,I swear to God. This dumb nigger bitch. I have...,1
47690,Yea fuck you RT @therealexel: IF YOURE A NIGGE...,1


In [7]:
train_size = 32000

train_data = dataset[0:train_size]
test_data = dataset[train_size:]

train_text = train_data['tweet_text']
train_labels = train_data['cyberbullying_type']

test_text = test_data['tweet_text']
test_labels = test_data['cyberbullying_type']

,tweet_text,cyberbullying_type
0,"In other words #katandandre, your food was cra...",3
1,Why is #aussietv so white? #MKR #theblock #ImA...,3
2,@XochitlSuckkks a classy whore? Or more red ve...,3
3,"@Jason_Gio meh. :P thanks for the heads up, b...",3
4,@RudhoeEnglish This is an ISIS account pretend...,3
...,...,...
31995,Dude wtf are you talking about. Standing up to...,0
31996,I just love going to my hometown and going on ...,0
31997,king princess- let’s non lesbian friends use t...,0
31998,I got bullied for watching Pewdiepie in high s...,0


In [13]:
tokenizer = get_tokenizer('basic_english')

def yield_tokens(data):
    for text in data['tweet_text']:
        yield tokenizer(text)

vocab = build_vocab_from_iterator(yield_tokens(dataset), specials=["<unk>"])

vocab.set_default_index(vocab["<unk>"])

vocab_size = len(vocab)
batch_size = 32
embedding_dim = 100
epochs = 25
num_classes = 6

69768

In [ ]:
class TextDataset(Dataset):
    def __init__(self, data):
        self.text = data['tweet_text'].tolist()
        self.labels = data['cyberbullying_type'].tolist()
        
    def __len__(self):
        return len(self.text)
    
    def __getitem__(self, idx):
        text = self.text[idx]
        label = self.labels[idx]
        tokenized_text = tokenize(text)
        
        return tokenized_text, label

train_text_dataset =  TextDataset(train_text, train_labels)
valid_text_dataset = TextDataset(test_text, test_labels)

train_loader = DataLoader(train_text_dataset, batch_size=batch_size, shuffle=True)
test_loader = DataLoader(test_text_dataset, batch_size=batch_size, shuffle=True)


In [ ]:
class TextClassificationModel(nn.Module):
    def __init__(self, vocab_size, embed_dim, num_class):
        super(TextClassificationModel, self).__init__()
        self.embedding = nn.EmbeddingBag(vocab_size, embed_dim, sparse=False)
        self.fc = nn.Linear(embed_dim, num_class)
        self.ini_weights()
    
    def init_weights(self):
        initrange = 0.5
        self.embedding.weight.data.uniform_(-initrange, initrange)
        self.fc.weight.data.uniform_(-initrange, initrange)
        self.fc.bias.data.zero_()
        
    def forward(self, text, offsets):
        embedded = self.embedding(text, offsets)
        output = self.fc(embedded)
        return output

device = 'cuda'
comment_classifier = TextClassificationModel(vocab_size, embed_dim, num_classes).to(device)

In [ ]:
def train(dataloader, model):
    model.train()
    total_acc, total_count = 0, 0
    log_interval = 500
    start_time = time.time()
    
    for idx, 
    